In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Induction of the polarity of a word from context word

so far: given the embedding of a word, learn to predict its polarity
e.g. given "hope" = positive, learn to predict it form the embedding

### binary classification using a word context
now: given a  word as context, learn to predict if they have the same polarity
e.g. given "joy", combine it with "hope", predict whether the have the same polarity

idea: attention would improve it over the single case approach

finding: yes, if we use BERT, no if we use fasttext and a simple architecture
BERT reaches 97% accuracy

training data: pairs of word with same (1) or differenct (0) polarity
we have many trainging data now, almost n x n, where n is the size of the lexicon (we don't combine a word with itself)

### classifing pairs for "same polarity"

* we have pos,pos pairs (neg,neg and neut,neut)

In [2]:
import json

path="/content/gdrive/My Drive/ml20/"

with open(path+"polex_embeddings.txt","r") as infile:
    emb2 = json.load(infile)

with open(path+"polex_embedding_index.txt","r") as infile:
    id_to_pol = json.load(infile)
    
with open(path+"polex_embedding_words.txt","r") as infile:
    id_to_word = json.load(infile)
    
with open(path+"polex_embedding_pollist.txt","r") as infile:
    posnegneut = json.load(infile)


emb=list(emb2['1'])

pos=posnegneut["1"]
neg=posnegneut["0"]
neut=posnegneut["2"]

In [3]:
from random import randint

def getpair(data,a,b,pol,c):
    pairs=[]

    for i in range(0,len(a)-1):
        for _ in range(0,c):
            j=randint(0,len(b)-1)
            if a[i] in data and b[j] in data:  # i!=j is possible, but doesn't harm
                pairs.append((b[j],a[i],pol))
    return pairs


def mappol(pol):
    if pol==0:
        return [1,0,0]
    elif pol==1:
        return [0,1,0]
    else:
        return [0,0,1]

In [4]:
from sklearn.utils import shuffle

def datasplit():
  ids=list(id_to_pol.keys())   # list of ids of words pointing to their embedding vector
  ids=[int(i) for  i in ids]
  ids=shuffle(ids)   

  split=int(len(ids)*0.7)   # 70% to train
    
  train=ids[:split]

  test=ids[split+1:]        

  # create pairs for training and testing
  #depth
  d=5
  data=train
  pair_pos = getpair(data,pos,pos,1,d)   # 15 = number of pair per noun
  pair_neg = getpair(data,neg,neg,0,d)
  pair_neut = getpair(data,neut,neut,2,d)
    
  data=test
  ypair_pos = getpair(data,pos,pos,1,d)  
  ypair_neg = getpair(data,neg,neg,0,d)
  ypair_neut = getpair(data,neut,neut,2,d)

  
  same_pol=pair_pos+pair_neg +pair_neut
  ysame_pol=ypair_pos+ypair_neg +ypair_neut

  X_test2=ysame_pol # +ydiff_pol
  X_test3=shuffle(X_test2,random_state=0)

  X_test = [[emb[a],emb[b]] for (a,b,_) in X_test3]
  y_test =[mappol(pol) for (_,_,pol) in X_test3]

  X_train2=same_pol #+diff_pol
  X_train3=ids=shuffle(X_train2,random_state=0)                    

  X =[[emb[a],emb[b]] for (a,b,_) in X_train3]
  y =[mappol(pol) for (_,_,pol) in X_train3]

  return X,y,X_test,y_test

In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class attention(nn.Module):
    def __init__(self,  fdim):
        super().__init__()

        self.tokeys = nn.Linear(fdim,fdim)
        self.toqueries = nn.Linear(fdim,fdim)
        self.tovalues = nn.Linear(fdim,fdim)
                    
    def forward(self,x):

        Q=self.toqueries(x)
        K=self.tokeys(x)
        V=self.tovalues(x)
        QK = Q @ K.transpose(1,2)
        QK_softmax = F.softmax(QK, dim=-1)
        
        weighted_values = QK_softmax @ V

        return weighted_values

In [9]:
from torch.utils.data import Dataset, DataLoader

class SentimentData(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.labels = labels
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):     
        X = self.data[index]
        y = self.labels[index]

        return torch.tensor(X,dtype=torch.float32,requires_grad=True), torch.tensor(y,dtype=torch.float32)

In [24]:
class transformer3(nn.Module):
    
    def __init__(self, k):
        super().__init__()

        self.attention = attention(k)
        self.norm = nn.LayerNorm(k)

        self.ff = nn.Sequential(
          nn.Linear(k , k),
         # nn.ReLU(),
          nn.Linear(k, 3))
            
    def forward(self, x):
                
        x = self.attention(x)
        x = x.unsqueeze(dim=0)
        x=self.norm(x)   

        x=self.ff(x)

        x= F.max_pool2d(x,kernel_size=(2,1)) 

        m=nn.Softmax(dim=3)
        x=m(x)
        
        return x.squeeze(dim=2)


class transformer2(nn.Module):
    
    def __init__(self, k):
        super().__init__()

        self.attention = attention(k)
        self.norm = nn.LayerNorm(k)

        self.ff = nn.Sequential(
          nn.Linear(k, 4 * k),
          nn.ReLU(),
          nn.Linear(4 * k, k ),
          nn.ReLU(),
          nn.Linear(k , k),
          nn.ReLU(),
          nn.Linear(k, k),
          nn.ReLU(),
          nn.Linear(k, 3))
            
    def forward(self, x):
                
        x = self.attention(x)
        x = x.unsqueeze(dim=0)
        x=self.norm(x)   

        x=self.ff(x)

        x= F.max_pool2d(x,kernel_size=(2,1)) 

        m=nn.Softmax(dim=3)
        x=m(x)
        
        return x.squeeze(dim=2)


class transformer(nn.Module):
    
    def __init__(self, k):
        super().__init__()

        self.attention = attention(k)
        self.norm = nn.LayerNorm(k)
        
        self.ff = nn.Sequential(
          nn.Linear(k, 4 * k),
          nn.Dropout(),   # default=0.5
          nn.ReLU(),
          nn.Dropout(),
          nn.Linear(4 * k, k ),
          nn.Dropout(0.25),
          nn.ReLU(),
          nn.Dropout(0.25),
          nn.Linear(k , k),
          nn.Dropout(0.25),
          nn.ReLU(),
          nn.Dropout(0.25),
          nn.Linear(k, k),
          nn.Dropout(0.25),
          nn.ReLU(),
          nn.Dropout(0.25),
          nn.Linear(k, 3),
          nn.Dropout(0.25))
            
    def forward(self, x):
                
        x = self.attention(x)
        x = x.unsqueeze(dim=0)
        x=self.norm(x)   

        x=self.ff(x)

        x= F.avg_pool2d(x,kernel_size=(2,1)) 

        m=nn.Softmax(dim=3)
        x=m(x)
        
        return x.squeeze(dim=2)
    


In [7]:
def training(epochs):
  for epoch in range(epochs):
      for input,label in  dataloader:
          optimizer.zero_grad()       
      
          outputs = trans(input.to(device))   
          label=label.to(device)
  
          loss = loss_func(outputs, label.unsqueeze(dim=0))
        
          loss.backward()
          optimizer.step()


def evaluate(X_test,y_test):
  tp,all=0,0

  for input,label in zip(X_test,y_test):
      input=torch.tensor(input,dtype=torch.float)
    
      input=input.to(device)
      output = trans(input.unsqueeze(0))
      predict=torch.argmax(output)
      predict=predict.cpu()
 #   print(predict.detach().numpy(), np.argmax(label))
  
      if predict.detach().numpy() == np.argmax(label):
          tp+=1
      all+=1
  return tp/all


In [25]:
# run it

batch_size=50
epochs=1

acc=0
loss_func = torch.nn.MSELoss()

folds=5  # crossvalidation

for i in range(folds):
  print("fold",i+1)
  X,y,X_test,y_test = datasplit()

  data=SentimentData(X,y)
  dataloader=DataLoader(dataset=data,batch_size=batch_size,shuffle=True)
  trans=transformer(300)
  trans.to(device)
  trans.train()    # do apply dropout

  optimizer = torch.optim.Adam(trans.parameters(), weight_decay=1e-5, lr=1e-4)

  print("learning")
  training(epochs)   
  print("testing")
  trans.eval()     # do not apply dropout
  acc2=evaluate(X_test,y_test)   
  print(acc2)
  acc+=acc2

print("mean accuracy=", acc/folds)

fold 1
learning
testing
0.8958875585632483
fold 2
learning
testing
0.8879581151832461
fold 3
learning
testing
0.8750634840020315
fold 4
learning
testing
0.8722724853645556
fold 5
learning
testing
0.8915478615071283
mean accuracy= 0.8845459009240418
